In [6]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
urlBase = "https://www.meczyki.pl"
urlTeams = "https://www.meczyki.pl/ekstraklasa,119,liga"

In [9]:
def allPlayers():
    first = True
    temp_source = requests.get(urlTeams).text
    temp_soup = BeautifulSoup(temp_source, "lxml")
    teams = temp_soup.find_all("a",attrs={'class':"team-url"})
    for team in teams:
            link = urlBase+team['href']
            temp_source = requests.get(link).text
            temp_soup = BeautifulSoup(temp_source, "lxml")
            players = temp_soup.find_all('td', attrs={"class": "align-left team"})
            for player in players:
                playerID = player.find("a")['href']
                link = urlBase + playerID
                if playerID == team['href']:
                    playerID = player.find("a").text
                    player = {"id":playerID}
                    playersDF = playersDF.append(player, ignore_index=True)
                    continue
                    
                temp_source = requests.get(link).text
                temp_soup = BeautifulSoup(temp_source, "lxml")
                playerName = temp_soup.find_all("div",attrs={"class": "list-el-name"})
                playerValue = temp_soup.find_all("div",attrs={"class": "list-el-value"})
                imie_nazwisko = temp_soup.find("h1").text.split()
                player = {"id":playerID,"imie": imie_nazwisko[0], "nazwisko":imie_nazwisko[1]}
                playerData = {}
                for j in range(len(playerName)):
                    name = playerName[j].text
                    value = playerValue[j].text.strip()
                    if j == 2:
                        value = value.split()[2]
                        value = datetime.strptime(value, "(%d.%m.%Y)")
                        name = "Data urodzenia"
                    if j == 4 or j == 6:
                        value = value.split()[0]
                        if value == "cm" or value == "kg":
                            value = None
                        else:
                            value = int(value)
                    playerData[name] = value
                player.update(playerData)
                if(first):
                    first = False
                    playersDF = pd.DataFrame(columns=player.keys())
                playersDF = playersDF.append(player, ignore_index=True)   
    return playersDF

In [10]:
Players = allPlayers()

In [37]:
Players.to_csv(r'PlayerListTest.csv', index = False,sep=';')

In [17]:
Players.head(5)

,id,imie,nazwisko,Obecny klub,Urodzony w,Data urodzenia,Pozycja,Wzrost,Noga,Waga
0,"/zawodnik-a-boruc,724",Artur,Boruc,Legia Warszawa,Siedlce,1980-02-20,Bramkarz,193,Prawa,87
1,"/zawodnik-w-muzyk,491228",Wojciech,Muzyk,Legia Warszawa,Suwałki,1998-11-07,Bramkarz,195,Brak,83
2,Artur Boruc,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,"/zawodnik-r-cierzniak,12549",Radosław,Cierzniak,Legia Warszawa,Szamocin,1983-04-24,Bramkarz,183,Prawa,81
4,"/zawodnik-c-miszta,509793",Cezary,Miszta,Legia Warszawa,Łuków,2001-10-30,Bramkarz,192,Brak,None
